### ¿Qué genera este script?

Para cada sitio y fecha, el total de usuarios registrados (usuarios.usuariosRegistrados)

In [1]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [2]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [3]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### Definiendo indice principal con fecha de hoy e indice de control

In [4]:
indice = parametros.usuarios_mintic_concat_index
indice_control = parametros.usuarios_mintic_control

### Función para generar JSON compatible con ElactiSearch

In [5]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud, COD_ISOid_Beneficiario).

In [6]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud','COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()
    

* Se valida latitud y longitud. Luego se calcula campo location
* Se renombran los campos de semilla

In [7]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return '4.596389' if y=='lat' else '-74.074639'

In [8]:
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))
datos_semilla['usuarios.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['usuarios.location']=datos_semilla['usuarios.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

datos_semilla = datos_semilla.rename(columns={'lugar_cod' : 'usuarios.centroDigitalUsuarios'
                                            , 'nombre_municipio': 'usuarios.nombreMunicipio'
                                            , 'nombre_departamento' : 'usuarios.nombreDepartamento'
                                            , 'nombre_centro_pob': 'usuarios.localidad'
                                            , 'nombreSede' : 'usuarios.nomCentroDigital'
                                            , 'energiadesc' : 'usuarios.sistemaEnergia'
                                            , 'COD_ISO' : 'usuarios.codISO'
                                            , 'id_Beneficiario' : 'usuarios.idBeneficiario'})
datos_semilla.fillna('', inplace=True)

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [9]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
               "_source": ["usuarios.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia_usuarios_registro"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['usuarios.fechaControl']
except:
    pass
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01 00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-08-10 00:00:00


### Asociando datos de usuarios

* Se realiza lectura por día procesado
* lugar_cod se usa para cruzar con semilla por site_id
* creado es la fecha de creación del usuario
* tipodoc y documento corresponden a datos del usuario. Se usa para calcular totales

In [10]:
def traeRegistros(fecha_max,fecha_tope):
    total_docs = 10000
    response = es.search(
        index= parametros.ohmyfi_d_u_index,
        body={
                "_source": ['lugar_cod','tipodoc', 'documento', 'creado']
                  , "query": {
                  "range": {
                    "creado": {
                      "gte": fecha_max.split(' ')[0]+' 00:00:00',
                      "lt": fecha_tope.split(' ')[0]+' 23:59:59'
                      #"gte": "2021-05-26 10:00:00",
                      #"lt": "2021-05-26 15:00:00"  
                    }
                  }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

### Se realiza consulta de datos

* Se calcula rango en base a la fecha de control. Para este caso es de un día.
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha actual.

In [13]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_registro = traeRegistros(fecha_max_mintic,fecha_tope_mintic)

if datos_registro is None or datos_registro.empty:
    while (datos_registro is None or datos_registro.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        datos_speed = traeRegistros(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

# Total registros  a indice

la lista use_these_keys se usa para referenciar cuales campos del dataframe irán al indice final. si los datos no se declaran en este, no se insertarán

In [14]:
use_these_keys = ['usuarios.fecha'
                  , 'usuarios.siteID'
                  , 'usuarios.usuariosRegistrados'
                  , 'usuarios.anyo'
                  , 'usuarios.mes'
                  , 'usuarios.dia'
                  , 'usuarios.nomCentroDigital'
                  , 'usuarios.codISO'
                  , 'usuarios.localidad'
                  , 'usuarios.nombreDepartamento'
                  , 'usuarios.sistemaEnergia'
                  , 'usuarios.nombreMunicipio'
                  , 'usuarios.idBeneficiario'
                  , 'usuarios.location'
                  , 'nombreDepartamento'
                    , 'nombreMunicipio'
                    , 'idBeneficiario'
                    , 'fecha'
                    , 'anyo'
                    , 'mes'
                    , 'dia'
                  , '@timestamp']

def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'UsuRegistro-'+ str(document['usuarios.siteID']) + '-' + str(document['usuarios.fecha'])}",
                "_source": filterKeys(document)
            }

Para cada sitio y fecha, se cuentan los documentos de usuario. Esto resulta en el total de usuarios registrados para esa fecha

In [15]:
def invalid_values(df, values=['']):
    return df.loc[df['usuarios.location'].isin(values)]

In [16]:
try:
    
    datos_registro['creado'] = datos_registro['creado'].str[0:10]
    datos_registro['tipo_mas_doc'] = datos_registro['tipodoc'] + datos_registro['documento']
    datos_registro= datos_registro.rename(columns={'creado' : 'usuarios.fecha'
                                                  ,'lugar_cod':'site_id'})

    datos_registro = datos_registro[["usuarios.fecha","site_id","tipo_mas_doc"]].groupby(["usuarios.fecha","site_id"]).agg(['count']).reset_index()
    datos_registro.columns = datos_registro.columns.droplevel(1)
    datos_registro= datos_registro.rename(columns={'tipo_mas_doc' : 'usuarios.usuariosRegistrados'})
    datos_registro["usuarios.anyo"] = datos_registro["usuarios.fecha"].str[0:4]
    datos_registro["usuarios.mes"] = datos_registro["usuarios.fecha"].str[5:7]
    datos_registro["usuarios.dia"] = datos_registro["usuarios.fecha"].str[8:10]
    datos_registro = pd.merge(datos_registro,  datos_semilla, on='site_id', how='inner')
    datos_registro.rename(columns={'site_id': 'usuarios.siteID'
                               }, inplace=True)
    
    datos_registro['nombreDepartamento'] = datos_registro['usuarios.nombreDepartamento']
    datos_registro['nombreMunicipio'] = datos_registro['usuarios.nombreMunicipio']
    datos_registro['idBeneficiario'] = datos_registro['usuarios.idBeneficiario']
    datos_registro['fecha'] = datos_registro['usuarios.fecha']
    datos_registro['anyo'] = datos_registro['usuarios.anyo']
    datos_registro['mes'] = datos_registro['usuarios.mes']
    datos_registro['dia'] = datos_registro['usuarios.dia']
    
    datos_registro['@timestamp'] = now.isoformat()
    
    salida = helpers.bulk(es, doc_generator(datos_registro))
    print("Fecha: ", now,"- Datos Registro usuarios en indice principal:",salida[0])
except Exception as e:
    print("Fecha: ", now,"- Nada que insertar en indice principal",e)

Fecha:  2021-08-10 08:26:54.610237 - Datos Registro usuarios en indice principal: 128


## Actualiza fecha control

* Se actualiza la fecha de control. Si el calculo supera la fecha actual, se asocia esta ultima.

In [17]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")#[0:15] + '0:00'    

if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:10] + ' 00:00:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:10] + ' 00:00:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia_usuarios_registro',
        body = { 'jerarquia_usuarios_registro': 'usuarios_registro','usuarios.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)

actualizada fecha control de ejecucion: 2021-08-10 00:00:00
